<a href="https://colab.research.google.com/github/puneat/Audio_Sentiment/blob/master/Copy_of_female_25GBRAM_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
# Import libraries 
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import glob 
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os
import sys
import warnings
# ignore warnings 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other  
from tqdm import tqdm, tqdm_pandas
import scipy
from scipy.stats import skew
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import sys
import IPython.display as ipd  # To play sound in the notebook
import warnings
# ignore warnings 
if not sys.warnoptions:
    warnings.simplefilter("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### **LOADING DATA**

In [ ]:
MALE = "/gdrive/My Drive/Audio_Sentiment/new3/male/"
FEMALE = "/gdrive/My Drive/Audio_Sentiment/new3/female/"


# Run one example 
dir_list = os.listdir(MALE)
dir_list[0:5]

['1272-135031-0015.flac',
 '1272-135031-0013.flac',
 '1272-135031-0014.flac',
 '1272-135031-0012.flac',
 '1272-135031-0010.flac']

In [ ]:
# Get the data location for SAVEE
dir_list = os.listdir(MALE)

# parse the filename to get the emotions
emotion=[]
path = []
for i in dir_list:
    emotion.append('male')
    path.append(MALE + i)
    
# Now check out the label count distribution 
MALE_df = pd.DataFrame(emotion, columns = ['labels'])
MALE_df['source'] = 'MALE'
MALE_df = pd.concat([MALE_df, pd.DataFrame(path, columns = ['path'])], axis = 1)
MALE_df.labels.value_counts()

male    1329
Name: labels, dtype: int64

In [ ]:
# Get the data location for SAVEE
dir_list = os.listdir(FEMALE)

# parse the filename to get the emotions
emotion=[]
path = []
for i in dir_list:
    emotion.append('female')
    path.append(FEMALE + i)
    
# Now check out the label count distribution 
FEMALE_df = pd.DataFrame(emotion, columns = ['labels'])
FEMALE_df['source'] = 'FEMALE'
FEMALE_df = pd.concat([FEMALE_df, pd.DataFrame(path, columns = ['path'])], axis = 1)
FEMALE_df.labels.value_counts()

female    1374
Name: labels, dtype: int64

In [ ]:
df = pd.concat([MALE_df,FEMALE_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("/gdrive/My Drive/Audio_Sentiment/gender3.csv",index=False)

female    1374
male      1329
Name: labels, dtype: int64


### **AUGMENTATION**

In [ ]:
ref3 = pd.read_csv("/gdrive/My Drive/Audio_Sentiment/gender3.csv")
ref3.head()

,labels,source,path
0,male,MALE,/gdrive/My Drive/Audio_Sentiment/new3/male/127...
1,male,MALE,/gdrive/My Drive/Audio_Sentiment/new3/male/127...
2,male,MALE,/gdrive/My Drive/Audio_Sentiment/new3/male/127...
3,male,MALE,/gdrive/My Drive/Audio_Sentiment/new3/male/127...
4,male,MALE,/gdrive/My Drive/Audio_Sentiment/new3/male/127...


In [ ]:
################################################################################
''' Adding White Noise '''

def noise(data):
    #Can take any distribution from https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html
    noise_amp = 0.05*np.random.uniform()*np.amax(data)   # more noise reduce the value to 0.5
    data = data.astype('float64') + noise_amp * np.random.normal(size=data.shape[0])
    return data

################################################################################
''' Random Shifting '''

def shift(data):
    s_range = int(np.random.uniform(low=-5, high = 5)*1000)  #default at 500
    return np.roll(data, s_range)

################################################################################
''' Streching the Sound. This expands the audio size '''

def stretch(data, rate=0.8):
    data = librosa.effects.time_stretch(data, rate)
    return data

################################################################################    
''' Pitch Tuning '''
def pitch(data, sample_rate):
    bins_per_octave = 12
    pitch_pm = 2
    pitch_change =  pitch_pm * 2*(np.random.uniform())   
    data = librosa.effects.pitch_shift(data.astype('float64'), 
                                      sample_rate, n_steps=pitch_change, 
                                      bins_per_octave=bins_per_octave)
    return data

################################################################################    
''' Dynamic Change'''

def dyn_change(data):
    #Random Value Change
    dyn_change = np.random.uniform(low=-0.5 ,high=7)
    return (data * dyn_change)

################################################################################
''' Speed and Pitch Tuning '''

def speedNpitch(data):
    length_change = np.random.uniform(low=0.8, high = 1)
    speed_fac = 1.2  / length_change 
    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)
    minlen = min(data.shape[0], tmp.shape[0])
    data *= 0
    data[0:minlen] = tmp[0:minlen]
    return data

In [ ]:
'''Extracting the MFCC feature as an image (Matrix format).'''
def prepare_data(df, n):
    X_speednpitch = np.empty(shape=(df.shape[0], n, 259, 1))
    X_noise = np.empty(shape=(df.shape[0], n, 259, 1))
    X_pitch = np.empty(shape=(df.shape[0], n, 259, 1))
    X_shift = np.empty(shape=(df.shape[0], n, 259, 1))
    X_dyn_change = np.empty(shape=(df.shape[0], n, 259, 1))
    X_ref = np.empty(shape=(df.shape[0], n, 259, 1))
    input_length = sampling_rate * audio_duration
    
    cnt = 0
    train_cnt=0
    test_cnt=0
    for fname in tqdm(df.path):
        file_path = fname
        data, _ = librosa.load(file_path, sr=sampling_rate
                               ,res_type="kaiser_fast"
                               ,duration=3
                               ,offset=0.5
                              )

        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, int(input_length) - len(data) - offset), "constant")

        # Augmentation 
            data_speednpitch = speedNpitch(data)
            data_noise=noise(data)
            data_shift=shift(data)
            data_pitch=pitch(data,sampling_rate)
            data_dyn_change=dyn_change(data)

        # MFCC extraction
        # if cnt%5==0:
        #     MFCC = librosa.feature.mfcc(data, sr=sampling_rate, n_mfcc=n_mfcc)
        #     MFCC = np.expand_dims(MFCC, axis=-1)
        #     X_test[test_cnt,] = MFCC
        #     test_cnt=test_cnt+1
        # elif cnt%5!=0:
            MFCC = librosa.feature.mfcc(data_speednpitch, sr=sampling_rate, n_mfcc=n_mfcc)
            MFCC = np.expand_dims(MFCC, axis=-1)
            X_speednpitch[train_cnt,] = MFCC

            MFCC = librosa.feature.mfcc(data_noise, sr=sampling_rate, n_mfcc=n_mfcc)
            MFCC = np.expand_dims(MFCC, axis=-1)
            X_noise[train_cnt,] = MFCC

            MFCC = librosa.feature.mfcc(data_pitch, sr=sampling_rate, n_mfcc=n_mfcc)
            MFCC = np.expand_dims(MFCC, axis=-1)
            X_pitch[train_cnt,] = MFCC

            MFCC = librosa.feature.mfcc(data_shift, sr=sampling_rate, n_mfcc=n_mfcc)
            MFCC = np.expand_dims(MFCC, axis=-1)
            X_shift[train_cnt,] = MFCC

            MFCC = librosa.feature.mfcc(data_dyn_change, sr=sampling_rate, n_mfcc=n_mfcc)
            MFCC = np.expand_dims(MFCC, axis=-1)
            X_dyn_change[train_cnt,] = MFCC

            MFCC = librosa.feature.mfcc(data, sr=sampling_rate, n_mfcc=n_mfcc)
            MFCC = np.expand_dims(MFCC, axis=-1)
            X_ref[train_cnt,] = MFCC
        train_cnt=train_cnt+1  
        #else:
            # Log-melspectogram
            # melspec = librosa.feature.melspectrogram(data, n_mels = n_melspec)   
            # logspec = librosa.amplitude_to_db(melspec)
            # logspec = np.expand_dims(logspec, axis=-1)
            # X[cnt,] = logspec
            
        cnt += 1

    return X_ref, X_speednpitch, X_pitch, X_shift, X_noise, X_dyn_change#, X_test

In [ ]:
sampling_rate=44100
audio_duration=3
n_mfcc = 50
df_ref, df_speednpitch, df_pitch, df_shift, df_noise, df_dyn_change = prepare_data(ref3, n = n_mfcc)

100%|██████████| 2703/2703 [40:01<00:00,  1.13it/s]


In [ ]:
del result

In [ ]:
print(df_noise.shape,df_speednpitch.shape, df_pitch.shape, df_dyn_change.shape, df_shift.shape,df_ref.shape)

(2703, 50, 259, 1) (2703, 50, 259, 1) (2703, 50, 259, 1) (2703, 50, 259, 1) (2703, 50, 259, 1) (2703, 50, 259, 1)


In [ ]:
result3=np.concatenate((df_noise,df_speednpitch,df_pitch,df_shift,df_dyn_change,df_ref),axis=0)

In [ ]:
result3.shape

(16218, 50, 259, 1)

In [ ]:
np.save('/gdrive/My Drive/Audio_Sentiment/gender3_2d_50_aug.npy',result3)

In [ ]:
result3=np.load('/gdrive/My Drive/Audio_Sentiment/gender3_2d_50_aug.npy')

In [ ]:
result2=np.load('/gdrive/My Drive/Audio_Sentiment/gender2_2d_50_aug.npy')

In [ ]:
result1=np.load('/gdrive/My Drive/Audio_Sentiment/gender_2d_50_aug.npy')

In [ ]:
final=np.concatenate((result1,result2,result3),axis=0)

In [ ]:
del result3,result2,result1

### **MODEL**

In [ ]:
# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM, Reshape
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling2D
from keras.utils import np_utils, to_categorical
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from keras.layers import (Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation, Input, Dense)

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other  
from tqdm import tqdm, tqdm_pandas
import scipy
from scipy.stats import skew
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import sys
import IPython.display as ipd  # To play sound in the notebook
import warnings
# ignore warnings 
if not sys.warnoptions:
    warnings.simplefilter("ignore")

Using TensorFlow backend.


In [ ]:
ref1 = pd.read_csv("/gdrive/My Drive/Audio_Sentiment/gender.csv")
ref1.head()

,labels,source,path
0,male,MALE,/gdrive/My Drive/Audio_Sentiment/new/male/61-7...
1,male,MALE,/gdrive/My Drive/Audio_Sentiment/new/male/61-7...
2,male,MALE,/gdrive/My Drive/Audio_Sentiment/new/male/61-7...
3,male,MALE,/gdrive/My Drive/Audio_Sentiment/new/male/1089...
4,male,MALE,/gdrive/My Drive/Audio_Sentiment/new/male/61-7...


In [ ]:
ref2 = pd.read_csv("/gdrive/My Drive/Audio_Sentiment/gender2.csv")
ref2.head()

,labels,source,path
0,male,MALE,/gdrive/My Drive/Audio_Sentiment/new2/male/693...
1,male,MALE,/gdrive/My Drive/Audio_Sentiment/new2/male/693...
2,male,MALE,/gdrive/My Drive/Audio_Sentiment/new2/male/693...
3,male,MALE,/gdrive/My Drive/Audio_Sentiment/new2/male/693...
4,male,MALE,/gdrive/My Drive/Audio_Sentiment/new2/male/693...


In [ ]:
frames=[ref1,ref1,ref1,ref1,ref1,ref1,ref2,ref2,ref2,ref2,ref2,ref2,ref3,ref3,ref3,ref3,ref3,ref3];
df_label=pd.concat(frames);

In [ ]:
nclass = 2
inp = Input(shape=(50,259,1))  #2D matrix of 30 MFCC bands by 216 audio length.

# X=Reshape((50,259),name='reshape_input')(inp)
# X = LSTM(units=259,dropout=0.2,return_sequences=True,activation='relu',stateful=False,input_shape=(50,259))(X)
# X=Reshape((50,259,1),name='reshape')(X)  #2D matrix of 30 MFCC bands by 216 audio length.

x = Convolution2D(64, (4,10), padding="same")(inp)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPool2D()(x)
# x = Dropout(rate=0.2)(x)
    
x = Convolution2D(64, (4,10), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPool2D()(x)
# x = Dropout(rate=0.2)(x)
    
x = Convolution2D(64, (4,10), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPool2D()(x)

x = Convolution2D(64, (4,10), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPool2D()(x)

# x = Dropout(rate=0.2)(x)
    
# x = Dropout(rate=0.2)(x)
x= AveragePooling2D()(x)
x = Flatten()(x)
# x = Dense(64)(x)
# # x = Dropout(rate=0.2)(x)
# x = BatchNormalization()(x)
# x = Activation("relu")(x)
# x = Dropout(rate=0.2)(x)

# x = Dense(64)(x)
# # x = Dropout(rate=0.2)(x)
# x = BatchNormalization()(x)
# x = Activation("relu")(x)
# x = Dropout(rate=0.2)(x)
# x=Reshape((128,1),name='reshape')(x)  #2D matrix of 30 MFCC bands by 216 audio length.
# x = LSTM(units=50,dropout=0.2,return_sequences=False,activation='relu',stateful=False)(x)
    
out = Dense(nclass, activation='softmax')(x)
model = models.Model(inputs=inp, outputs=out)
    
opt = optimizers.Adam(0.001,amsgrad=True)
model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['accuracy'])
print('done')

done


In [ ]:
df_label.shape, final.shape

((31488, 3), (31488, 50, 259, 1))

In [ ]:
# Split between train and test 
X_train, X_test, y_train, y_test = train_test_split(final
,df_label.labels, test_size=1, shuffle=True, random_state=42)

In [ ]:
# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

# Normalization as per the standard NN process
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

In [ ]:
model_json = model.to_json()
with open("/gdrive/My Drive/Audio_Sentiment/models/json/gender_newdata_5.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights t o HDF5
model.save_weights("/gdrive/My Drive/Audio_Sentiment/models/gender_newdata_5.hdf5")
print("Saved model to disk")

Saved model to disk


In [ ]:
model_checkpoint = ModelCheckpoint('/gdrive/My Drive/Audio_Sentiment/models/hdf5/gender_newdata_5.hdf5',verbose=2,period=20)
history=model.fit(X_train,y_train,initial_epoch=60,epochs=80,batch_size=16,verbose=2,shuffle=True,callbacks=[model_checkpoint])

Epoch 61/80
 - 52s - loss: 8.2452e-06 - accuracy: 1.0000
Epoch 62/80
 - 52s - loss: 9.3263e-06 - accuracy: 1.0000
Epoch 63/80
 - 52s - loss: 2.1174e-05 - accuracy: 1.0000
Epoch 64/80
 - 52s - loss: 1.0683e-05 - accuracy: 1.0000
Epoch 65/80
 - 52s - loss: 9.8571e-06 - accuracy: 1.0000
Epoch 66/80
 - 52s - loss: 9.7242e-06 - accuracy: 1.0000
Epoch 67/80
 - 52s - loss: 1.0520e-05 - accuracy: 1.0000
Epoch 68/80
 - 52s - loss: 8.4295e-06 - accuracy: 1.0000
Epoch 69/80
 - 52s - loss: 8.9032e-06 - accuracy: 1.0000
Epoch 70/80
 - 52s - loss: 8.1439e-06 - accuracy: 1.0000
Epoch 71/80
 - 52s - loss: 7.5609e-06 - accuracy: 1.0000
Epoch 72/80
 - 52s - loss: 8.2313e-06 - accuracy: 1.0000
Epoch 73/80
 - 52s - loss: 1.0702e-05 - accuracy: 1.0000
Epoch 74/80
 - 52s - loss: 8.1640e-06 - accuracy: 1.0000
Epoch 75/80
 - 52s - loss: 7.2532e-06 - accuracy: 1.0000
Epoch 76/80
 - 52s - loss: 6.7906e-06 - accuracy: 1.0000
Epoch 77/80
 - 52s - loss: 6.9104e-06 - accuracy: 1.0000
Epoch 78/80
 - 52s - loss: 6.83

OSError: ignored